<a href="https://colab.research.google.com/github/shirinsitara/IMLO/blob/26-working-model/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [132]:
#transform = transforms.ToTensor()
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.ToTensor(),          # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

In [133]:
#train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

In [134]:
from torchvision.datasets import Flowers102

# Load the training data
train_data = Flowers102(root='/flower_data', split='train', download=True, transform=transform)

# Load the validation data
val_data = Flowers102(root='/flower_data', split='val', download=True, transform=transform)

# Load the test data
test_data = Flowers102(root='/flower_data', split='test', download=True, transform=transform)


In [135]:
train_data

Dataset Flowers102
    Number of datapoints: 1020
    Root location: /flower_data
    split=train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [136]:
test_data

Dataset Flowers102
    Number of datapoints: 6149
    Root location: /flower_data
    split=test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [137]:
val_data

Dataset Flowers102
    Number of datapoints: 1020
    Root location: /flower_data
    split=val
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [138]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
val_loader = DataLoader(val_data, batch_size=10, shuffle=False)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [139]:
conv1 = nn.Conv2d(3, 16, 3, 2, 2)
conv2 = nn.Conv2d(16, 32, 3, 2, 2)
conv3 = nn.Conv2d(32, 64, 3, 2, 2)
conv4 = nn.Conv2d(64, 128, 3, 2, 2)

In [140]:
for i, (X_Train, y_train) in enumerate(train_data):
  break

In [141]:
X_Train.shape

torch.Size([3, 224, 224])

In [142]:
x = X_Train.view(1,3,224,224)

In [143]:
x = F.relu(conv1(x))
x = F.max_pool2d(x,2,2,1)
x = F.relu(conv2(x))
x = F.max_pool2d(x, 2, 2, 1)
x = F.relu(conv3(x))
x = F.max_pool2d(x, 2, 2, 1)

In [144]:
x.shape

torch.Size([1, 64, 5, 5])

In [145]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,16,3,2,2)
    self.conv2 = nn.Conv2d(16,32,3,2,2)
    self.conv3 = nn.Conv2d(32,64, 3, 2, 2)
    # Fully Connected Layer
    self.fc1 = nn.Linear(5*5*64, 1200)
    self.fc2 = nn.Linear(1200, 720)
    self.fc3 = nn.Linear(720, 102)


  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X,2,2, 1)
    # Second Pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2, 2, 1)
    X = F.relu(self.conv3(X))
    X = F.max_pool2d(X, 2, 2, 1)

    # Re-View to flatten it out
    X = X.view(-1, 64*5*5) # negative one so that we can vary the batch size

    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1)

In [146]:
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
  (fc1): Linear(in_features=1600, out_features=1200, bias=True)
  (fc2): Linear(in_features=1200, out_features=720, bias=True)
  (fc3): Linear(in_features=720, out_features=102, bias=True)
)

In [147]:
# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

In [148]:
import time
start_time = time.time()

# Create Variables To Tracks Things
epochs = 30
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0


  # Train
  for b,(X_train, y_train) in enumerate(train_loader):
    b+=1 # start our batches at 1
    y_pred = model(X_train) # get predicted values from the training set. Not flattened 2D
    loss = criterion(y_pred, y_train) # how off are we? Compare the predictions to correct answers in y_train

    predicted = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions. Indexed off the first point
    batch_corr = (predicted == y_train).sum() # how many we got correct from this batch. True = 1, False=0, sum those up
    trn_corr += batch_corr # keep track as we go along in training.

    # Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    # Print out some results
    # if b%10 == 0:
    #   print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)


  # Test
  with torch.no_grad(): #No gradient so we don't update our weights and biases with test data
    for b,(X_test, y_test) in enumerate(val_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1] # Adding up correct predictions
      tst_corr += (predicted == y_test).sum() # T=1 F=0 and sum away


  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)

  train_accuracy = trn_corr / len(train_loader.dataset) * 100
  test_accuracy = tst_corr / len(val_loader.dataset) * 100

  print(f'Epoch {i+1} - Training Accuracy: {train_accuracy:.2f}% - Validation Accuracy: {test_accuracy:.2f}%')



current_time = time.time()
total = current_time - start_time
print(f'Training Took: {total/60} minutes!')

Epoch 1 - Training Accuracy: 0.78% - Validation Accuracy: 0.98%
Epoch 2 - Training Accuracy: 0.78% - Validation Accuracy: 1.08%
Epoch 3 - Training Accuracy: 1.27% - Validation Accuracy: 1.86%
Epoch 4 - Training Accuracy: 1.08% - Validation Accuracy: 2.16%
Epoch 5 - Training Accuracy: 1.18% - Validation Accuracy: 2.06%
Epoch 6 - Training Accuracy: 1.27% - Validation Accuracy: 2.45%
Epoch 7 - Training Accuracy: 2.16% - Validation Accuracy: 3.24%
Epoch 8 - Training Accuracy: 2.65% - Validation Accuracy: 3.24%
Epoch 9 - Training Accuracy: 3.63% - Validation Accuracy: 5.10%
Epoch 10 - Training Accuracy: 4.80% - Validation Accuracy: 4.90%
Epoch 11 - Training Accuracy: 5.39% - Validation Accuracy: 7.94%
Epoch 12 - Training Accuracy: 8.14% - Validation Accuracy: 6.86%
Epoch 13 - Training Accuracy: 10.20% - Validation Accuracy: 9.22%
Epoch 14 - Training Accuracy: 12.25% - Validation Accuracy: 10.69%
Epoch 15 - Training Accuracy: 15.69% - Validation Accuracy: 13.14%
Epoch 16 - Training Accuracy:

In [150]:
train_losses = [tl.item() for tl in train_losses]
plt.plot(train_losses, label="Training Loss")
plt.plot(test_losses, label="Validation Loss")
plt.title("Loss at Epoch")
plt.legend()

AttributeError: 'float' object has no attribute 'item'